In [2]:
import pandas as pd
import mlflow
import mlflow.sklearn
import xgboost as xgb
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
import time

In [4]:
# Load dataset
df = pd.read_excel("../data/ENB2012_data.xlsx")

# Define features and targets
X = df.drop(columns=['Y1', 'Y2'])  
y = df[['Y1', 'Y2']]  
# Split dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Initialize and train XGBoost model
xgb_model = xgb.XGBRegressor(objective='reg:squarederror', n_estimators=200, learning_rate=0.1, random_state=42)
xgb_model.fit(X_train, y_train)

# Make predictions
y_pred_xgb = xgb_model.predict(X_test)

start_time = time.time()  # Start timer
xgb_model.fit(X_train, y_train)  # Train the model
end_time = time.time()  # End timer
training_time = end_time - start_time

# Evaluate performance for both Y1 (Heating) and Y2 (Cooling)
mae_y1_xgb = mean_absolute_error(y_test.iloc[:, 0], y_pred_xgb[:, 0])
mse_y1_xgb = mean_squared_error(y_test.iloc[:, 0], y_pred_xgb[:, 0])
r2_y1_xgb = r2_score(y_test.iloc[:, 0], y_pred_xgb[:, 0])

mae_y2_xgb = mean_absolute_error(y_test.iloc[:, 1], y_pred_xgb[:, 1])
mse_y2_xgb = mean_squared_error(y_test.iloc[:, 1], y_pred_xgb[:, 1])
r2_y2_xgb = r2_score(y_test.iloc[:, 1], y_pred_xgb[:, 1])

# Print results
print("XGBoost Regressor Results:")
print(f"Heating Load (Y1) - MAE: {mae_y1_xgb}, MSE: {mse_y1_xgb}, R2: {r2_y1_xgb}")
print(f"Cooling Load (Y2) - MAE: {mae_y2_xgb}, MSE: {mse_y2_xgb}, R2: {r2_y2_xgb}")
print(f"Training Time: {training_time:.2f} seconds")

XGBoost Regressor Results:
Heating Load (Y1) - MAE: 0.26015774352829174, MSE: 0.16348012315575197, R2: 0.9984315806927794
Cooling Load (Y2) - MAE: 0.5237115686590023, MSE: 0.7954066867255357, R2: 0.9914156010506407
Training Time: 0.25 seconds


In [5]:
# Set MLflow Tracking URI 
mlflow.set_tracking_uri("http://127.0.0.1:5000")  # Localhost MLflow server

# Create or set experiment
experiment_name = "Energy_Efficiency_Prediction"
mlflow.set_experiment(experiment_name)

# Log experiment in MLflow
with mlflow.start_run() as run:
    mlflow.log_param("n_estimators", 100)
    mlflow.log_metric("MAE_Y1", mae_y1_xgb)
    mlflow.log_metric("MSE_Y1", mse_y1_xgb)
    mlflow.log_metric("R2_Y1", r2_y1_xgb)
    mlflow.log_metric("MAE_Y2", mae_y2_xgb)
    mlflow.log_metric("MSE_Y2", mse_y2_xgb)
    mlflow.log_metric("R2_Y2", r2_y2_xgb)

    # Log training time
    mlflow.log_metric("training_time_seconds", training_time)

    mlflow.sklearn.log_model(xgb_model, "xgboost_model")

    # Register the model
    model_uri = f"runs:/{run.info.run_id}/xgboost_model"
    mlflow.register_model(model_uri, "XGBoostModel")

2025/03/09 12:16:08 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.
Successfully registered model 'XGBoostModel'.
2025/03/09 12:16:08 INFO mlflow.store.model_registry.abstract_store: Waiting up to 300 seconds for model version to finish creation. Model name: XGBoostModel, version 1


🏃 View run smiling-eel-314 at: http://127.0.0.1:5000/#/experiments/1/runs/d571ac5467c44b6288453de745fbf9d4
🧪 View experiment at: http://127.0.0.1:5000/#/experiments/1


Created version '1' of model 'XGBoostModel'.
